In [108]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import env

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

from acquire import get_zillow_data

In [109]:
df= get_zillow_data()
df.head()

,Unnamed: 0,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,parcelid.1,logerror,transactiondate,buildingclasstypeid.1,buildingclassdesc,storytypeid.1,storydesc,typeconstructiontypeid.1,typeconstructiondesc,parcelid.2
0,0,2061546,11289917,1.0,NaN,NaN,2.0,3.0,NaN,6.0,...,11289917,-0.362001,2017-06-23,NaN,NaN,NaN,NaN,NaN,NaN,11289917
1,1,1834372,11705026,NaN,NaN,NaN,1.0,2.0,NaN,6.0,...,11705026,-0.146056,2017-06-30,NaN,NaN,NaN,NaN,NaN,NaN,11705026
2,2,1923117,14269464,NaN,NaN,NaN,3.0,4.0,NaN,NaN,...,14269464,0.021085,2017-06-01,NaN,NaN,NaN,NaN,NaN,NaN,14269464
3,3,2121349,11389003,NaN,NaN,NaN,2.0,3.0,NaN,6.0,...,11389003,-0.325393,2017-06-01,NaN,NaN,NaN,NaN,NaN,NaN,11389003
4,4,2093710,11967869,NaN,NaN,NaN,1.0,2.0,NaN,5.0,...,11967869,-0.005566,2017-06-29,NaN,NaN,NaN,NaN,NaN,NaN,11967869


In [110]:
# Let's figure out how much data is missing where
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    pct_missing = num_missing / rows
    cols_missing = pd.DataFrame({'number_missing_rows': num_missing, 'percent_rows_missing': pct_missing})
    return cols_missing

In [111]:
#checking which columns have the most null rows
nulls_by_column = nulls_by_col(df)
nulls_by_column.sort_values(by="percent_rows_missing", ascending=False, inplace=True)
nulls_by_column           

,number_missing_rows,percent_rows_missing
buildingclassdesc,20394,1.000000
buildingclasstypeid.1,20394,1.000000
buildingclasstypeid,20394,1.000000
finishedsquarefeet15,20391,0.999853
storytypeid.1,20383,0.999461
...,...,...
fips,0,0.000000
bedroomcnt,0,0.000000
bathroomcnt,0,0.000000
parcelid,0,0.000000


In [152]:
def clean_zillow(cached=True):
    '''This function acquires and prepares the zillow data from a local csv, default. Passing cached=False acquires fresh data from sql and writes to csv.'''
    # use my aquire function to read data into a df from a csv file
    df = get_zillow_data()
    # drop duplicates
    df.drop_duplicates(inplace=True)
    # drop duplicate columns and remove columns with more than 50% nulls
    df = df.drop(columns=['unitcnt','propertylandusedesc','heatingorsystemdesc','propertyzoningdesc','heatingorsystemtypeid','taxdelinquencyflag','taxdelinquencyyear','yardbuildingsqft17','finishedsquarefeet50','finishedfloor1squarefeet','fireplacecnt','threequarterbathnbr','pooltypeid7','poolcnt','numberofstories','airconditioningdesc','garagetotalsqft','garagecarcnt','regionidneighborhood','hashottuborspa','pooltypeid2','poolsizesum','pooltypeid10','typeconstructiontypeid','typeconstructiondesc','architecturalstyledesc','finishedsquarefeet6','fireplaceflag','yardbuildingsqft26','finishedsquarefeet13','storytypeid','storydesc','basementsqft','finishedsquarefeet15','buildingclassdesc','architecturalstyletypeid','airconditioningtypeid','buildingclasstypeid','buildingqualitytypeid','decktypeid','architecturalstyletypeid.1','airconditioningtypeid.1','heatingorsystemtypeid.1','propertylandusetypeid.1','buildingclasstypeid.1', 'storytypeid.1', 'typeconstructiontypeid.1','id.1','Unnamed: 0','calculatedbathnbr', 'fips', 'latitude', 'longitude', 'regionidcounty', 'roomcnt', 'yearbuilt', 'assessmentyear', 'propertycountylandusecode', 'propertylandusetypeid', 'parcelid.2','parcelid.1'])
    #removing columns
    df.replace(',','', regex=True, inplace=True)
    #handling nan's
    df.fillna(df.mean())
    return df

In [153]:
df=clean_zillow()

In [154]:
df.head()

,id,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,finishedsquarefeet12,fullbathcnt,lotsizesquarefeet,rawcensustractandblock,regionidcity,regionidzip,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate
0,2061546,11289917,2.0,3.0,1458.0,1458.0,2.0,8284.0,6.037901e+07,5534.0,97318.0,108890.0,136104.0,27214.0,2319.90,6.037901e+13,-0.362001,2017-06-23
1,1834372,11705026,1.0,2.0,1421.0,1421.0,1.0,6707.0,6.037232e+07,12447.0,96018.0,11982.0,35606.0,23624.0,543.69,6.037232e+13,-0.146056,2017-06-30
2,1923117,14269464,3.0,4.0,2541.0,2541.0,3.0,4975.0,6.059064e+07,38032.0,96958.0,434887.0,880456.0,445569.0,9819.72,6.059064e+13,0.021085,2017-06-01
3,2121349,11389003,2.0,3.0,1650.0,1650.0,2.0,7300.0,6.037703e+07,396550.0,96037.0,165000.0,614000.0,449000.0,7673.19,6.037703e+13,-0.325393,2017-06-01
4,2093710,11967869,1.0,2.0,693.0,693.0,1.0,2908.0,6.037185e+07,12447.0,96023.0,82416.0,274237.0,191821.0,3267.47,6.037185e+13,-0.005566,2017-06-29


In [155]:
#checking null count after making clean data frame
nulls_by_column = nulls_by_col(df)
nulls_by_column.sort_values(by="percent_rows_missing", ascending=False, inplace=True)
nulls_by_column           

,number_missing_rows,percent_rows_missing
lotsizesquarefeet,2263,0.110964
regionidcity,366,0.017946
finishedsquarefeet12,112,0.005492
fullbathcnt,84,0.004119
censustractandblock,78,0.003825
calculatedfinishedsquarefeet,53,0.002599
structuretaxvaluedollarcnt,33,0.001618
regionidzip,14,0.000686
taxamount,1,0.000049
taxvaluedollarcnt,0,0.000000


In [147]:
# #making my split, train, test data
train_validate, test = train_test_split(df, test_size=.2, 
                                         random_state=42,
                                           )
train, validate = train_test_split(train_validate, test_size=.3, 
                                  random_state=42,
                                         ) 

In [148]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20394 entries, 0 to 20393
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            20394 non-null  int64  
 1   parcelid                      20394 non-null  int64  
 2   bathroomcnt                   20394 non-null  float64
 3   bedroomcnt                    20394 non-null  float64
 4   calculatedfinishedsquarefeet  20341 non-null  float64
 5   finishedsquarefeet12          20282 non-null  float64
 6   fullbathcnt                   20310 non-null  float64
 7   lotsizesquarefeet             18131 non-null  float64
 8   rawcensustractandblock        20394 non-null  float64
 9   regionidcity                  20028 non-null  float64
 10  regionidzip                   20380 non-null  float64
 11  unitcnt                       12752 non-null  float64
 12  structuretaxvaluedollarcnt    20361 non-null  float64
 13  t

In [125]:
#combining my split, train, test data and my clean data into one dataframe
def prep_zillow_data():
    '''This function will return a data frame holding both my clean data and the split/train/test data.'''
    df = clean_zillow()
    train_validate, test = train_test_split(df, test_size=.2, random_state=42)
    train, validate = train_test_split(train_validate, 
                                       test_size=.3, 
                                       random_state=42)
    X_train = train.drop(columns='logerror')
    X_validate = validate.drop(columns='logerror')
    X_test = test.drop(columns='logerror')

    y_train = train['logerror']
    y_validate = validate['logerror']
    y_test = test['logerror']
    return train, validate, test

In [129]:
def scaled_data(train, validate, test):

    train = train.drop(['logerror','calculatedfinishedsquarefeet'], axis=1)
    validate = validate.drop(['logerror','calculatedfinishedsquarefeet'], axis=1)
    test = test.drop(['logerror','calculatedfinishedsquarefeet'], axis=1)

    # 1. Create the Scaling Object
    scaler = sklearn.preprocessing.StandardScaler()

    # 2. Fit to the train data only
    scaler.fit(train)

    # 3. use the object on the whole df
    # this returns an array, so we convert to df in the same line
    train_scaled = pd.DataFrame(scaler.transform(train))
    validate_scaled = pd.DataFrame(scaler.transform(validate))
    test_scaled = pd.DataFrame(scaler.transform(test))

    # the result of changing an array to a df resets the index and columns
    # for each train, validate, and test, we change the index and columns back to original values

    # Train
    train_scaled.index = train.index
    train_scaled.columns = train.columns

    # Validate
    validate_scaled.index = validate.index
    validate_scaled.columns = validate.columns

    # Test
    test_scaled.index = test.index
    test_scaled.columns = test.columns

    return train_scaled, validate_scaled, test_scaled

In [156]:
!git add 'prepare.ipynb'

In [157]:
!git commit -m 'updates'

[main afa699b] updates
 1 file changed, 136 insertions(+), 142 deletions(-)


In [158]:
!git push

Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 4 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 2.05 KiB | 2.05 MiB/s, done.
Total 3 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:Gabby-B-B/clustering-project.git
   0d5055e..afa699b  main -> main
